In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import string

In [2]:
# template_ids = ['3267']
template_ids = ['13868']
template_labels = [1]
template = {11:[2878], 12:[42], 13:[760], 14:[734, 3765], 15:[522, 13511],
       16:[154, 429], 17:[6], 18:[1251, 151, 152], 19:[2], 20:[5]}

template_url_prefix = 'https://memes.tw/wtf?template='
meme_url_prefix = 'https://memes.tw/wtf/'
# page_limit = 25 + 1
# page_limit = 3 + 1
size_per_class = 500

data = {}

In [3]:
def get_text(label_idx):
  data[label_idx] = {}
  for template_id in template[label_idx]:
    page_idx = 1
    while len(data[label_idx]) < size_per_class:
      print(label_idx, template_id, len(data[label_idx]))
      template_url = template_url_prefix + str(template_id) + '&page=' + str(page_idx)
      r = requests.get(template_url)
      r.encoding='utf-8'
      soup = BeautifulSoup(r.text, "html.parser")
      mydivs = soup.find_all("div", {"class": "sensitive-content"})
      if not mydivs: # No more memes
        break
      for mydiv in mydivs:
        meme_url = meme_url_prefix + mydiv['data-id']
        r2 = requests.get(meme_url)
        r2.encoding='utf-8'
        soup2 = BeautifulSoup(r2.text, "html.parser")
        mydivs2 = soup2.find_all("div", {"class": "text-muted text-truncate"})

        if mydivs2:
          text = mydivs2[0].string.strip()[5:]
        else:
          text = soup2.title.string
        # print(text)
        if text and (text[0] not in string.ascii_letters) and (text[:2] != "文字"):
          # data.append([text, label])
          data[label_idx][text] = label_idx
      page_idx += 1

In [5]:
import threading

t = []
for label_idx in template.keys():
  t.append(threading.Thread(target=get_text, args=(label_idx,)))
  t[-1].start()

for thread in t:
  thread.join()
print("end")

11 2878 0
12 42 0
13 1126 0
14 734 015
16 8005 0
 154 017 6 0

18 12519  0
20 2 0
2898 0
11 2878 18
17 6 16
12 42 19
19 2 19
13 1126 19
14 734 18
18 125 18
20 2898 20
16 154 17
15 8005 19
17 6 35
11 2878 38
14 734 38
19 2 38
12 42 38
13 1126 39
18 125 37
20 2898 40
16 154 35
15 8005 36
15 8005 41
15 8006 41
17 6 53
14 734 57
12 42 55
13 1126 59
11 2878 57
20 2898 60
16 154 54
18 125 57
19 2 57
15 8006 60
17 6 72
11 2878 71
14 734 77
13 1126 78
20 2898 80
12 42 75
16 154 71
19 2 74
18 125 76
15 8006 76
17 6 91
11 2878 89
14 734 95
20 2898 99
13 1126 96
19 2 94
12 42 95
16 154 86
18 125 96
15 8006 95
11 2878 109
17 6 110
14 734 113
20 2898 119
19 2 113
12 42 112
13 1126 116
16 154 104
18 125 115
15 8006 113
17 6 130
14 734 132
11 2878 127
19 2 133
20 2898 138
12 42 130
13 1126 134
18 125 134
16 154 124
15 8006 133
17 6 147
11 2878 146
14 734 151
19 2 152
20 2898 157
1513 8006 141
 1126 154
12 42 148
18 125 152
15 13288 141
16 154 143
11 2878 166
17 6 165
14 734 170
20 2898 176
19 2 171
1

In [9]:
with open('data_11_20.csv', 'w', newline='', encoding="utf-8") as csvfile:
    csvfile.write('\ufeff')
    writer = csv.writer(csvfile)
    for label_idx in template.keys():
      if label_idx == 13 or label_idx == 15 or label_idx == 20:
        continue
      writer.writerows(data[label_idx].items())

In [8]:
for label_idx in template.keys():
  print(label_idx, len(data[label_idx]))

11 511
12 515
13 291
14 500
15 341
16 468
17 480
18 381
19 451
20 195
